In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
(include-book "http-json" 
              :ttags ((:quicklisp) (:quicklisp.dexador) (:http-json)))
(include-book "kestrel/json-parser/parse-json-file" :dir :system)
(include-book "std/strings/cat" :dir :system)
(include-book "std/strings/explode-nonnegative-integer" :dir :system)
(include-book "std/util/bstar" :dir :system)
(include-book "std/util/define" :dir :system)
(include-book "centaur/fty/top" :dir :system)


TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

TTAG NOTE (for included book): Adding ttag :QUICKLISP.DEXADOR from book /home/acl2/books/quicklisp/dexador.

TTAG NOTE (for included book): Adding ttag :HTTP-JSON from book /workspaces/verified-agent/src/http-json.

Summary
Form:  ( INCLUDE-BOOK "http-json" ...)
Rules: NIL
Time:  0.38 seconds (prove: 0.00, print: 0.00, other: 0.38)
 "/workspaces/verified-agent/src/http-json.lisp"

Summary
Form:  ( INCLUDE-BOOK "kestrel/json-parser/parse-json-file" ...)
Rules: NIL
Time:  0.06 seconds (prove: 0.00, print: 0.00, other: 0.06)
 "/home/acl2/books/kestrel/json-parser/parse-json-file.lisp"

Summary
Form:  ( INCLUDE-BOOK "std/strings/cat" ...)
Rules: NIL
Time:  0.07 seconds (prove: 0.00, print: 0.00, other: 0.07)
 "/home/acl2/books/std/strings/cat.lisp"

Summary
Form:  ( INCLUDE-BOOK "std/strings/explode-nonnegative-integer" ...)
Rules: NIL
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.0

In [3]:
;; Default MCP server endpoint (mcp-proxy or uvx mcp-server-http)
(defconst *mcp-default-endpoint* 
  "http://localhost:8000/mcp")


Summary
Form:  ( DEFCONST *MCP-DEFAULT-ENDPOINT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MCP-DEFAULT-ENDPOINT*


In [4]:
(defconst *mcp-connect-timeout* 10)   ; seconds
(defconst *mcp-read-timeout* 60)      ; seconds (higher for ACL2 operations)


Summary
Form:  ( DEFCONST *MCP-CONNECT-TIMEOUT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MCP-CONNECT-TIMEOUT*

Summary
Form:  ( DEFCONST *MCP-READ-TIMEOUT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MCP-READ-TIMEOUT*


In [5]:
(fty::defprod mcp-connection
  ((endpoint stringp :default "")
   (transport-session-id stringp :default "")
   (acl2-session-id stringp :default ""))
  :tag :mcp-connection
  :layout :tree)

Form:  ( TABLE XDOC ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-ORIG-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-TYPE-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( MAKE-EVENT (CONS ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( IN-THEORY (ENABLE ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( DEFLABEL FTY::DEFTYPES-BEFORE-TEMP-THMS ...)
Time:  0.00 seconds (prove: 0.00, pr

In [6]:
;; Predicate for valid connection (has non-empty endpoint and transport session)
(define mcp-connection-valid-p ((conn mcp-connection-p))
  :returns (valid booleanp)
  (and (not (equal (mcp-connection->endpoint conn) ""))
       (not (equal (mcp-connection->transport-session-id conn) ""))))


Since MCP-CONNECTION-VALID-P is non-recursive, its admission is trivial.
We observe that the type of MCP-CONNECTION-VALID-P is described by
the theorem 
(OR (EQUAL (MCP-CONNECTION-VALID-P CONN) T)
    (EQUAL (MCP-CONNECTION-VALID-P CONN)
           NIL)).

Computing the guard conjecture for MCP-CONNECTION-VALID-P....

The guard conjecture for MCP-CONNECTION-VALID-P is trivial to prove,
given the :compound-recognizer rule CONSP-WHEN-MCP-CONNECTION-P.  
MCP-CONNECTION-VALID-P is compliant with Common Lisp.

Summary
Form:  ( DEFUN MCP-CONNECTION-VALID-P ...)
Rules: ((:COMPOUND-RECOGNIZER CONSP-WHEN-MCP-CONNECTION-P))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM BOOLEANP-OF-MCP-CONNECTION-VALID-P
          (B* ((VALID (MCP-CONNECTION-VALID-P CONN)))
            (BOOLEANP VALID))
          :HINTS
          ((STD::RETURNSPEC-DEFAULT-DEFAULT-HINT 'MCP-CONNECTION-VALID-P
                                                 ID WORLD))
          :RULE-CLASSES :REWRI

In [7]:
;; Check if connection has a persistent ACL2 session
(define mcp-connection-has-acl2-session-p ((conn mcp-connection-p))
  :returns (has-session booleanp)
  (not (equal (mcp-connection->acl2-session-id conn) "")))


Since MCP-CONNECTION-HAS-ACL2-SESSION-P is non-recursive, its admission
is trivial.  We observe that the type of MCP-CONNECTION-HAS-ACL2-SESSION-P
is described by the theorem 
(OR (EQUAL (MCP-CONNECTION-HAS-ACL2-SESSION-P CONN)
           T)
    (EQUAL (MCP-CONNECTION-HAS-ACL2-SESSION-P CONN)
           NIL)).

Computing the guard conjecture for MCP-CONNECTION-HAS-ACL2-SESSION-P....

The guard conjecture for MCP-CONNECTION-HAS-ACL2-SESSION-P is trivial
to prove.  MCP-CONNECTION-HAS-ACL2-SESSION-P is compliant with Common
Lisp.

Summary
Form:  ( DEFUN MCP-CONNECTION-HAS-ACL2-SESSION-P ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM BOOLEANP-OF-MCP-CONNECTION-HAS-ACL2-SESSION-P
          (B* ((HAS-SESSION (MCP-CONNECTION-HAS-ACL2-SESSION-P CONN)))
            (BOOLEANP HAS-SESSION))
          :HINTS ((STD::RETURNSPEC-DEFAULT-DEFAULT-HINT
                       'MCP-CONNECTION-HAS-ACL2-SESSION-P
                       ID WORLD))
          :RU

In [8]:
(fty::defprod mcp-result
  ((has-error booleanp :default nil)
   (error stringp :default "")
   (content stringp :default ""))
  :tag :mcp-result
  :layout :tree)

Form:  ( TABLE XDOC ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( TABLE STD::DEFAULT-HINTS-TABLE ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-ORIG-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( DEFTHEORY FTY::DEFTYPES-TYPE-THEORY ...)
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Form:  ( MAKE-EVENT (CONS ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( IN-THEORY (ENABLE ...))
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
Form:  ( DEFLABEL FTY::DEFTYPES-BEFORE-TEMP-THMS ...)
Time:  0.00 seconds (prove: 0.00, pr

In [9]:
;; Quick check for success
(define mcp-result-ok-p ((result mcp-result-p))
  :returns (ok booleanp)
  (not (mcp-result->has-error result)))


Since MCP-RESULT-OK-P is non-recursive, its admission is trivial. 
We observe that the type of MCP-RESULT-OK-P is described by the theorem
(OR (EQUAL (MCP-RESULT-OK-P RESULT) T) (EQUAL (MCP-RESULT-OK-P RESULT) NIL)).

Computing the guard conjecture for MCP-RESULT-OK-P....

The guard conjecture for MCP-RESULT-OK-P is trivial to prove.  
MCP-RESULT-OK-P is compliant with Common Lisp.

Summary
Form:  ( DEFUN MCP-RESULT-OK-P ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM BOOLEANP-OF-MCP-RESULT-OK-P
          (B* ((OK (MCP-RESULT-OK-P RESULT)))
            (BOOLEANP OK))
          :HINTS ((STD::RETURNSPEC-DEFAULT-DEFAULT-HINT 'MCP-RESULT-OK-P
                                                        ID WORLD))
          :RULE-CLASSES :REWRITE)

ACL2 Warning [Non-rec] in ( DEFTHM BOOLEANP-OF-MCP-RESULT-OK-P ...):
A :REWRITE rule generated from BOOLEANP-OF-MCP-RESULT-OK-P will be
triggered only by terms containing the function symbol MCP-RESULT-OK

In [10]:
;; Serialize MCP tool call request to JSON-RPC 2.0
(defun serialize-mcp-tool-call (tool-name args-json request-id)
  (declare (xargs :guard (and (stringp tool-name)
                              (stringp args-json)
                              (natp request-id)))
           (ignore tool-name args-json request-id))
  (prog2$ (er hard? 'serialize-mcp-tool-call "Raw Lisp definition not installed?")
          "{}"))


Since SERIALIZE-MCP-TOOL-CALL is non-recursive, its admission is trivial.
We observe that the type of SERIALIZE-MCP-TOOL-CALL is described by
the theorem 
(STRINGP (SERIALIZE-MCP-TOOL-CALL TOOL-NAME ARGS-JSON REQUEST-ID)).

Computing the guard conjecture for SERIALIZE-MCP-TOOL-CALL....

The guard conjecture for SERIALIZE-MCP-TOOL-CALL is trivial to prove.
SERIALIZE-MCP-TOOL-CALL is compliant with Common Lisp.

Summary
Form:  ( DEFUN SERIALIZE-MCP-TOOL-CALL ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 SERIALIZE-MCP-TOOL-CALL


In [11]:
(defthm stringp-of-serialize-mcp-tool-call
  (stringp (serialize-mcp-tool-call tool-name args-json request-id)))


ACL2 Warning [Non-rec] in ( DEFTHM STRINGP-OF-SERIALIZE-MCP-TOOL-CALL
...):  A :REWRITE rule generated from STRINGP-OF-SERIALIZE-MCP-TOOL-CALL
will be triggered only by terms containing the function symbol 
SERIALIZE-MCP-TOOL-CALL, which has a non-recursive definition.  Unless
this definition is disabled, this rule is unlikely ever to be used.


Q.E.D.

The storage of STRINGP-OF-SERIALIZE-MCP-TOOL-CALL depends upon the
:type-prescription rule SERIALIZE-MCP-TOOL-CALL.

Summary
Form:  ( DEFTHM STRINGP-OF-SERIALIZE-MCP-TOOL-CALL ...)
Rules: ((:TYPE-PRESCRIPTION SERIALIZE-MCP-TOOL-CALL))
Warnings:  Non-rec
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 STRINGP-OF-SERIALIZE-MCP-TOOL-CALL


In [12]:
;; Parse MCP JSON-RPC response, extract result content or error
;; Returns (cons error-or-nil content-string)
(defun parse-mcp-response-raw (json)
  (declare (xargs :guard (stringp json))
           (ignore json))
  (prog2$ (er hard? 'parse-mcp-response-raw "Raw Lisp definition not installed?")
          (cons "Raw Lisp not loaded" "")))


Since PARSE-MCP-RESPONSE-RAW is non-recursive, its admission is trivial.
We observe that the type of PARSE-MCP-RESPONSE-RAW is described by
the theorem 
(AND (CONSP (PARSE-MCP-RESPONSE-RAW JSON))
     (NOT (TRUE-LISTP (PARSE-MCP-RESPONSE-RAW JSON)))).

Computing the guard conjecture for PARSE-MCP-RESPONSE-RAW....

The guard conjecture for PARSE-MCP-RESPONSE-RAW is trivial to prove.
PARSE-MCP-RESPONSE-RAW is compliant with Common Lisp.

Summary
Form:  ( DEFUN PARSE-MCP-RESPONSE-RAW ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 PARSE-MCP-RESPONSE-RAW


In [13]:
(defthm consp-of-parse-mcp-response-raw
  (consp (parse-mcp-response-raw json)))


ACL2 Warning [Non-rec] in ( DEFTHM CONSP-OF-PARSE-MCP-RESPONSE-RAW
...):  A :REWRITE rule generated from CONSP-OF-PARSE-MCP-RESPONSE-RAW
will be triggered only by terms containing the function symbol 
PARSE-MCP-RESPONSE-RAW, which has a non-recursive definition.  Unless
this definition is disabled, this rule is unlikely ever to be used.


Q.E.D.

The storage of CONSP-OF-PARSE-MCP-RESPONSE-RAW depends upon the :type-
prescription rule PARSE-MCP-RESPONSE-RAW.

Summary
Form:  ( DEFTHM CONSP-OF-PARSE-MCP-RESPONSE-RAW ...)
Rules: ((:TYPE-PRESCRIPTION PARSE-MCP-RESPONSE-RAW))
Warnings:  Non-rec
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 CONSP-OF-PARSE-MCP-RESPONSE-RAW


In [14]:
;; Wrapper that returns mcp-result
(define parse-mcp-response ((json stringp))
  :returns (result mcp-result-p)
  (b* ((raw (parse-mcp-response-raw json))
       (err (car raw))
       (content (if (stringp (cdr raw)) (cdr raw) "")))
    (make-mcp-result :has-error (if (stringp err) t nil)
                     :error (if (stringp err) err "")
                     :content content)))


Since PARSE-MCP-RESPONSE is non-recursive, its admission is trivial.
We observe that the type of PARSE-MCP-RESPONSE is described by the
theorem (CONSP (PARSE-MCP-RESPONSE JSON)).  We used the :type-prescription
rule MCP-RESULT.

Computing the guard conjecture for PARSE-MCP-RESPONSE....

The guard conjecture for PARSE-MCP-RESPONSE is trivial to prove, given
the :compound-recognizer rule BOOLEANP-COMPOUND-RECOGNIZER, primitive
type reasoning and the :type-prescription rule PARSE-MCP-RESPONSE-RAW.
PARSE-MCP-RESPONSE is compliant with Common Lisp.

Summary
Form:  ( DEFUN PARSE-MCP-RESPONSE ...)
Rules: ((:COMPOUND-RECOGNIZER BOOLEANP-COMPOUND-RECOGNIZER)
        (:FAKE-RUNE-FOR-TYPE-SET NIL)
        (:TYPE-PRESCRIPTION MCP-RESULT)
        (:TYPE-PRESCRIPTION PARSE-MCP-RESPONSE-RAW))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM MCP-RESULT-P-OF-PARSE-MCP-RESPONSE
          (B* ((RESULT (PARSE-MCP-RESPONSE JSON)))
            (MCP-RESULT-P RESULT))
          :

In [15]:
;; Serialize MCP initialize request to JSON-RPC 2.0
(defun serialize-mcp-initialize (request-id)
  (declare (xargs :guard (natp request-id))
           (ignore request-id))
  (prog2$ (er hard? 'serialize-mcp-initialize "Raw Lisp definition not installed?")
          "{}"))


Since SERIALIZE-MCP-INITIALIZE is non-recursive, its admission is trivial.
We observe that the type of SERIALIZE-MCP-INITIALIZE is described by
the theorem (STRINGP (SERIALIZE-MCP-INITIALIZE REQUEST-ID)).  

Computing the guard conjecture for SERIALIZE-MCP-INITIALIZE....

The guard conjecture for SERIALIZE-MCP-INITIALIZE is trivial to prove.
SERIALIZE-MCP-INITIALIZE is compliant with Common Lisp.

Summary
Form:  ( DEFUN SERIALIZE-MCP-INITIALIZE ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 SERIALIZE-MCP-INITIALIZE


In [16]:
(defthm stringp-of-serialize-mcp-initialize
  (stringp (serialize-mcp-initialize request-id)))


ACL2 Warning [Non-rec] in ( DEFTHM STRINGP-OF-SERIALIZE-MCP-INITIALIZE
...):  A :REWRITE rule generated from STRINGP-OF-SERIALIZE-MCP-INITIALIZE
will be triggered only by terms containing the function symbol 
SERIALIZE-MCP-INITIALIZE, which has a non-recursive definition.  Unless
this definition is disabled, this rule is unlikely ever to be used.


Q.E.D.

The storage of STRINGP-OF-SERIALIZE-MCP-INITIALIZE depends upon the
:type-prescription rule SERIALIZE-MCP-INITIALIZE.

Summary
Form:  ( DEFTHM STRINGP-OF-SERIALIZE-MCP-INITIALIZE ...)
Rules: ((:TYPE-PRESCRIPTION SERIALIZE-MCP-INITIALIZE))
Warnings:  Non-rec
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 STRINGP-OF-SERIALIZE-MCP-INITIALIZE


In [17]:
;; Parse MCP session ID from response headers alist
;; Returns session-id string or nil
(defun parse-mcp-session-id (headers-alist)
  (declare (xargs :guard (alistp headers-alist))
           (ignore headers-alist))
  (prog2$ (er hard? 'parse-mcp-session-id "Raw Lisp definition not installed?")
          nil))


Since PARSE-MCP-SESSION-ID is non-recursive, its admission is trivial.
We observe that the type of PARSE-MCP-SESSION-ID is described by the
theorem (EQUAL (PARSE-MCP-SESSION-ID HEADERS-ALIST) NIL).  

Computing the guard conjecture for PARSE-MCP-SESSION-ID....

The guard conjecture for PARSE-MCP-SESSION-ID is trivial to prove.
PARSE-MCP-SESSION-ID is compliant with Common Lisp.

Summary
Form:  ( DEFUN PARSE-MCP-SESSION-ID ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 PARSE-MCP-SESSION-ID


In [18]:
(defthm stringp-or-null-of-parse-mcp-session-id
  (or (null (parse-mcp-session-id headers))
      (stringp (parse-mcp-session-id headers)))
  :rule-classes (:rewrite :type-prescription))


ACL2 Observation in ( DEFTHM STRINGP-OR-NULL-OF-PARSE-MCP-SESSION-ID
...):  Our heuristics choose (PARSE-MCP-SESSION-ID HEADERS) as the
:TYPED-TERM.

ACL2 Warning [Non-rec] in ( DEFTHM STRINGP-OR-NULL-OF-PARSE-MCP-SESSION-ID
...):  A :REWRITE rule generated from STRINGP-OR-NULL-OF-PARSE-MCP-SESSION-ID
will be triggered only by terms containing the function symbols NULL
and PARSE-MCP-SESSION-ID, which have non-recursive definitions.  Unless
these definitions are disabled, this rule is unlikely ever to be used.


Q.E.D.

The storage of STRINGP-OR-NULL-OF-PARSE-MCP-SESSION-ID depends upon
the :type-prescription rule PARSE-MCP-SESSION-ID.

Summary
Form:  ( DEFTHM STRINGP-OR-NULL-OF-PARSE-MCP-SESSION-ID ...)
Rules: ((:TYPE-PRESCRIPTION PARSE-MCP-SESSION-ID))
Warnings:  Non-rec
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 STRINGP-OR-NULL-OF-PARSE-MCP-SESSION-ID


In [19]:
;; Build JSON object from key-value pairs (all string values)
(defun serialize-string-args (pairs)
  (declare (xargs :guard (alistp pairs))
           (ignore pairs))
  (prog2$ (er hard? 'serialize-string-args "Raw Lisp definition not installed?")
          "{}"))


Since SERIALIZE-STRING-ARGS is non-recursive, its admission is trivial.
We observe that the type of SERIALIZE-STRING-ARGS is described by the
theorem (STRINGP (SERIALIZE-STRING-ARGS PAIRS)).  

Computing the guard conjecture for SERIALIZE-STRING-ARGS....

The guard conjecture for SERIALIZE-STRING-ARGS is trivial to prove.
SERIALIZE-STRING-ARGS is compliant with Common Lisp.

Summary
Form:  ( DEFUN SERIALIZE-STRING-ARGS ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 SERIALIZE-STRING-ARGS


In [20]:
(defthm stringp-of-serialize-string-args
  (stringp (serialize-string-args pairs)))


ACL2 Warning [Non-rec] in ( DEFTHM STRINGP-OF-SERIALIZE-STRING-ARGS
...):  A :REWRITE rule generated from STRINGP-OF-SERIALIZE-STRING-ARGS
will be triggered only by terms containing the function symbol 
SERIALIZE-STRING-ARGS, which has a non-recursive definition.  Unless
this definition is disabled, this rule is unlikely ever to be used.


Q.E.D.

The storage of STRINGP-OF-SERIALIZE-STRING-ARGS depends upon the :type-
prescription rule SERIALIZE-STRING-ARGS.

Summary
Form:  ( DEFTHM STRINGP-OF-SERIALIZE-STRING-ARGS ...)
Rules: ((:TYPE-PRESCRIPTION SERIALIZE-STRING-ARGS))
Warnings:  Non-rec
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 STRINGP-OF-SERIALIZE-STRING-ARGS


In [21]:
(defttag :mcp-client)


TTAG NOTE: Adding ttag :MCP-CLIENT from the top level loop.
 :MCP-CLIENT


In [22]:
(include-raw "mcp-client-raw.lsp")



ACL2 !>>(PROGN!
         (SET-RAW-MODE T)
         (DEFUN RAW-COMPILE (NAME ERROR-ON-FAIL ON-FAIL STATE)
          (COND
           ((NOT (MEMBER :CLTL2 *FEATURES*))
            (COMPILE-FILE (EXTEND-PATHNAME (CBD) NAME STATE)))
           (T
            (HANDLER-CASE
             (COMPILE-FILE (EXTEND-PATHNAME (CBD) NAME STATE))
             (ERROR
              (CONDITION)
              (IF ERROR-ON-FAIL
               (LET ((CONDITION-STR (FORMAT NIL "~a" CONDITION)))
                (ER
                 HARD 'INCLUDE-RAW
                 "Compilation of ~x0 failed with the following message:~%~@1~%"
                 NAME CONDITION-STR))
               (EVAL
                (CONS
                  'LET
                  (CONS (CONS (CONS 'CONDITION
                                    (CONS (CONS 'QUOTE (CONS CONDITION 'NIL))
                                          'NIL))
                              'NIL)
                        (CONS '(DECLARE (IGNORABLE CONDITION))
          

In [23]:
;; Helper to check if HTTP status indicates success (2xx)
(define mcp-http-success-p ((status natp))
  :returns (success booleanp)
  (and (>= status 200)
       (< status 300)))


Since MCP-HTTP-SUCCESS-P is non-recursive, its admission is trivial.
We observe that the type of MCP-HTTP-SUCCESS-P is described by the
theorem 
(OR (EQUAL (MCP-HTTP-SUCCESS-P STATUS) T)
    (EQUAL (MCP-HTTP-SUCCESS-P STATUS)
           NIL)).

Computing the guard conjecture for MCP-HTTP-SUCCESS-P....

The guard conjecture for MCP-HTTP-SUCCESS-P is trivial to prove, given
the :compound-recognizer rule NATP-COMPOUND-RECOGNIZER.  MCP-HTTP-SUCCESS-P
is compliant with Common Lisp.

Summary
Form:  ( DEFUN MCP-HTTP-SUCCESS-P ...)
Rules: ((:COMPOUND-RECOGNIZER NATP-COMPOUND-RECOGNIZER))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)


ACL2 !>>(DEFTHM BOOLEANP-OF-MCP-HTTP-SUCCESS-P
          (B* ((SUCCESS (MCP-HTTP-SUCCESS-P STATUS)))
            (BOOLEANP SUCCESS))
          :HINTS ((STD::RETURNSPEC-DEFAULT-DEFAULT-HINT 'MCP-HTTP-SUCCESS-P
                                                        ID WORLD))
          :RULE-CLASSES :REWRITE)

ACL2 Warning [Non-rec] in ( DEFTHM BOOL

In [24]:
;; Global request ID counter
(defconst *mcp-request-id* 1)


Summary
Form:  ( DEFCONST *MCP-REQUEST-ID* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MCP-REQUEST-ID*


In [ ]:
(defun mcp-transport-connect (endpoint state)
  "Establish MCP transport session. Returns (mv err transport-session-id state)."
  (declare (xargs :guard (stringp endpoint)
                  :stobjs state
                  :mode :program))
  (b* ((request-json (serialize-mcp-initialize 1))
       (headers '(("Content-Type" . "application/json; charset=utf-8")
                  ("Accept" . "application/json, text/event-stream")))
       
       ((mv http-err body status-raw response-headers state)
        (post-json-with-headers endpoint request-json headers 
                                *mcp-connect-timeout* *mcp-read-timeout* state))
       
       (status (nfix status-raw))
       
       ((when http-err)
        (mv (str::cat "MCP transport HTTP error: " http-err) "" state))
       
       ((unless (mcp-http-success-p status))
        (mv (concatenate 'string "MCP transport HTTP status " 
                        (coerce (explode-nonnegative-integer status 10 nil) 'string)
                        ": " body) 
            "" state))
       
       (session-id (parse-mcp-session-id response-headers))
       
       ((unless session-id)
        (mv "MCP transport: no Mcp-Session-Id in response headers" "" state)))
    
    (mv nil session-id state)))

In [ ]:
(defun mcp-call-tool-raw (endpoint transport-session-id tool-name args state)
  "Call MCP tool using raw transport session. Returns (mv err content state)."
  (declare (xargs :guard (and (stringp endpoint)
                              (stringp transport-session-id)
                              (stringp tool-name)
                              (alistp args))
                  :stobjs state
                  :mode :program))
  (b* ((args-json (serialize-string-args args))
       (request-json (serialize-mcp-tool-call tool-name args-json *mcp-request-id*))
       (headers (list (cons "Content-Type" "application/json; charset=utf-8")
                      (cons "Accept" "application/json")
                      (cons "Mcp-Session-Id" transport-session-id)))
       
       ((mv http-err body status-raw state)
        (post-json endpoint request-json headers 
                   *mcp-connect-timeout* *mcp-read-timeout* state))
       
       (status (nfix status-raw))
       
       ((when http-err)
        (mv (str::cat "MCP HTTP error: " http-err) "" state))
       
       ((unless (mcp-http-success-p status))
        (mv (concatenate 'string "MCP HTTP status " 
                        (coerce (explode-nonnegative-integer status 10 nil) 'string)
                        ": " body) 
            "" state))
       
       (result (parse-mcp-response body)))
    
    (mv (if (mcp-result->has-error result)
            (mcp-result->error result)
          nil)
        (mcp-result->content result)
        state)))

In [ ]:
(defun mcp-connect (endpoint state)
  "Connect to MCP server, establish transport and ACL2 sessions.
   Returns (mv err connection state) where connection is mcp-connection-p."
  (declare (xargs :guard (stringp endpoint)
                  :stobjs state
                  :mode :program))
  (b* (;; Step 1: Establish MCP transport session
       ((mv transport-err transport-session-id state)
        (mcp-transport-connect endpoint state))
       
       ((when transport-err)
        (mv transport-err nil state))
       
       ;; Step 2: Start persistent ACL2 session
       ((mv acl2-err acl2-response state)
        (mcp-call-tool-raw endpoint transport-session-id "start_session"
                           (list (cons "name" "mcp-client-session"))
                           state))
       
       ;; Parse ACL2 session ID from response
       ;; Response format: "Session started successfully. ID: <uuid>"
       (acl2-session-id
        (if acl2-err
            ""  ; Fall back to one-shot mode if session start fails
          (b* ((id-pos (search "ID: " acl2-response))
               ((unless id-pos) "")
               (id-start (+ id-pos 4))
               ((unless (<= (+ id-start 36) (length acl2-response))) ""))
            (subseq acl2-response id-start (+ id-start 36)))))
       
       ;; Build connection object
       (conn (make-mcp-connection
              :endpoint endpoint
              :transport-session-id transport-session-id
              :acl2-session-id acl2-session-id)))
    
    (mv nil conn state)))

In [ ]:
(defun mcp-call-tool (conn tool-name args state)
  "Call an MCP tool. Returns (mv err content state)."
  (declare (xargs :guard (and (mcp-connection-p conn)
                              (stringp tool-name)
                              (alistp args))
                  :stobjs state
                  :mode :program))
  (mcp-call-tool-raw (mcp-connection->endpoint conn)
                     (mcp-connection->transport-session-id conn)
                     tool-name
                     args
                     state))

In [ ]:
(defun mcp-acl2-evaluate (conn code state)
  "Evaluate ACL2 code. Uses persistent session if available."
  (declare (xargs :guard (and (mcp-connection-p conn)
                              (stringp code))
                  :stobjs state
                  :mode :program))
  (let* ((session-id (mcp-connection->acl2-session-id conn))
         (args (if (equal session-id "")
                   (list (cons "code" code))
                 (list (cons "code" code)
                       (cons "session_id" session-id)))))
    (mcp-call-tool conn "evaluate" args state)))

In [ ]:
(defun mcp-acl2-admit (conn code state)
  "Test if ACL2 event would be admitted. Uses persistent session if available."
  (declare (xargs :guard (and (mcp-connection-p conn)
                              (stringp code))
                  :stobjs state
                  :mode :program))
  (let* ((session-id (mcp-connection->acl2-session-id conn))
         (args (if (equal session-id "")
                   (list (cons "code" code))
                 (list (cons "code" code)
                       (cons "session_id" session-id)))))
    (mcp-call-tool conn "admit" args state)))

In [ ]:
(defun mcp-acl2-prove (conn code state)
  "Prove ACL2 theorem. Uses persistent session if available."
  (declare (xargs :guard (and (mcp-connection-p conn)
                              (stringp code))
                  :stobjs state
                  :mode :program))
  (let* ((session-id (mcp-connection->acl2-session-id conn))
         (args (if (equal session-id "")
                   (list (cons "code" code))
                 (list (cons "code" code)
                       (cons "session_id" session-id)))))
    (mcp-call-tool conn "prove" args state)))

In [ ]:
(defun mcp-acl2-check-syntax (conn code state)
  "Check ACL2 code for syntax errors. Returns (mv err result state).
   err is non-nil if there are syntax errors."
  (declare (xargs :guard (and (mcp-connection-p conn)
                              (stringp code))
                  :stobjs state
                  :mode :program))
  ;; check_syntax doesn't need session_id - it's stateless
  (mcp-call-tool conn "check_syntax"
                 (list (cons "code" code))
                 state))

In [ ]:
(defun syntax-error-p (result)
  "Check if a check_syntax result indicates syntax errors.
   Returns T if the result starts with 'Syntax errors found:'."
  (declare (xargs :guard (stringp result)))
  (let ((prefix "Syntax errors found:"))
    (and (>= (length result) (length prefix))
         (equal (subseq result 0 (length prefix)) prefix))))

In [ ]:
(defun extract-syntax-error-detail (result)
  "Extract the actual error message from ACL2 syntax check output.
   Looks for 'Error:' line after the banner noise."
  (declare (xargs :guard (stringp result) :mode :program))
  (let ((error-pos (search "Error:" result)))
    (if (not error-pos)
        ;; No 'Error:' found, try to find ABORTING message
        (let ((abort-pos (search "ABORTING from raw Lisp" result)))
          (if (not abort-pos)
              "Syntax error (details unavailable)"
            ;; Find the Error: line after ABORTING
            (let ((after-abort (subseq result abort-pos (length result))))
              (let ((err-in-abort (search "Error:" after-abort)))
                (if (not err-in-abort)
                    "Syntax error (malformed s-expression)"
                  ;; Extract until next newline or ***
                  (let* ((err-start (+ abort-pos err-in-abort))
                         (rest (subseq result err-start (length result)))
                         (nl-pos (search (coerce '(#\Newline) 'string) rest))
                         (stars-pos (search "***" rest))
                         (end-pos (cond ((and nl-pos stars-pos) (min nl-pos stars-pos))
                                        (nl-pos nl-pos)
                                        (stars-pos stars-pos)
                                        (t (min 100 (length rest))))))
                    (subseq rest 0 end-pos)))))))
      ;; Found 'Error:' directly - extract to newline
      (let* ((rest (subseq result error-pos (length result)))
             (nl-pos (search (coerce '(#\Newline) 'string) rest))
             (end-pos (if nl-pos nl-pos (min 100 (length rest)))))
        (subseq rest 0 end-pos)))))

In [ ]:
(defun mcp-acl2-execute (conn code state)
  "Execute ACL2 code block via MCP. Handles multiple forms automatically.
   This is the main entry point for code execution - sends code to evaluate
   which handles multiple forms.
   Returns (mv err result state)."
  (declare (xargs :guard (and (mcp-connection-p conn)
                              (stringp code))
                  :stobjs state
                  :mode :program))
  ;; Just evaluate directly - the check_syntax tool is unreliable because
  ;; ACL2 doesn't have a true syntax-only checker. It runs the code with a
  ;; short timeout, which causes false positives on valid code that takes
  ;; more than 5 seconds to execute.
  (mcp-acl2-evaluate conn code state))

In [ ]:
(defun mcp-disconnect (conn state)
  "Clean up MCP connection, ending ACL2 session if active."
  (declare (xargs :guard (mcp-connection-p conn)
                  :stobjs state
                  :mode :program))
  (let ((session-id (mcp-connection->acl2-session-id conn)))
    (if (equal session-id "")
        (mv nil "No ACL2 session to end" state)
      (mcp-call-tool conn "end_session"
                     (list (cons "session_id" session-id))
                     state))))